In [ ]:
import sys
!{sys.executable} -m pip install pyttsx3
!{sys.executable} -m pip install pyttsx3 pywin32
!{sys.executable} -m pip install pyttsx3 numpy
!{sys.executable} -m pip install pyttsx3 opencv-python

In [2]:
import cv2

def count_vehicles(video_name):
    # Create a VideoCapture object and read the frames from an input file
    #video=cv2.VideoCapture('Videos\Traffic_Laramie_1.mp4')
    video=cv2.VideoCapture(video_name)

    # Check if the video opened successfully
    if (video.isOpened()== False): 
      print("Error opening video file")


    initial_frame = None
    counter = 0
    prev_frame_points = []
    tracking_cars = {}
    object_id = 0

    # We start an infinite loop and keep reading frames from the webcam until we press 'q'
    while True:
        check, frame = video.read()

        if check == True:

            # Gray conversion and noise reduction (smoothening)
            gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            blur_frame=cv2.GaussianBlur(gray_frame,(15,15), 0)


            # The first captured frame is the baseline image
            if initial_frame is None:
                initial_frame = blur_frame
                continue

            # The difference between the baseline and the new frame
            delta_frame=cv2.absdiff(initial_frame[320:400, 0:600],blur_frame[320:400, 0:600])
            # The difference (the delta_frame) is converted into a binary image
            # If a particular pixel value is greater than a certain threshold (specified by us here as 10),
            # it will be assigned the value for White (255) else Black(0)
            threshold_frame=cv2.threshold(delta_frame,10,255, cv2.THRESH_BINARY)[1]


            # The cv2.findContours() method we will identify all the contours in our image.
            # This method expects 3 parameters, (a) image, (b) contour retrieval mode and
            # (c) contour approximation method
            (contours,_)=cv2.findContours(threshold_frame,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

            #store the centre points of the cars for the current frame
            cur_frame_points = []

            for c in contours:
                # contourArea() method filters out any small contours
                # You can change this value
                if cv2.contourArea(c) < 1400:
                    continue
                (x, y, w, h)=cv2.boundingRect(c)
                cv2.rectangle(frame[300:], (x, y), (x+w, y+h), (0,255,0), 1)
                #get center point of objects
                cx = int((x+x+w)/2)
                cy = int((y+y+h)/2)
                cur_frame_points.append((cx,cy))

            if counter <= 2:
                for pt in cur_frame_points:
                    for pt2 in prev_frame_points:
                        distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])
                        if distance < 20:
                            tracking_objects[track_id] = pt
                            track_id += 1
            else:
                tracking_cars_copy = tracking_cars.copy()
                cur_frame_points_copy = cur_frame_points.copy()

                for object_id, pt2 in tracking_cars_copy.items():
                    object_exists = False
                    for pt in cur_frame_points_copy:
                        distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])
                        # Update IDs position
                        if distance < 20:
                            tracking_cars[object_id] = pt
                            object_exists = True
                            continue
                    # Remove IDs lost
                    if not object_exists:
                        tracking_objects.pop(object_id)

                # Add new IDs found
                for pt in cur_frame_points:
                    tracking_cars[track_id] = pt
                    track_id += 1

            cv2.putText(frame, 
                        str(object_id), 
                        (50, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, 
                        (0, 255, 255), 
                        2, 
                        cv2.LINE_4)

            # To better understand the application, we can visualise the different frames generated
            cv2.imshow('Video', frame)


            # Stop the program by pressing 'q'    
            if cv2.waitKey(1) == ord('q'):
                break

        else:
            break


    # After the loop release the video object
    video.release()

    # Destroy all the windows
    cv2.destroyAllWindows()

In [3]:
count_vehicles('Videos\Traffic_Laramie_1.mp4')

0
0


In [ ]:
count_vehicles('Videos\Traffic_Laramie_2.mp4')